In [1]:
!apt update && apt install -y curl git

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,315 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,655 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!nohup ollama serve > /dev/null 2>&1 &


In [4]:
!ollama pull llama2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   

In [5]:
OLLAMA_API_URL = "http://127.0.0.1:11434"

In [6]:
!pkill -f ngrok  # Kills all ngrok processes
!fuser -k 5000/tcp  # Kills any process using port 6000 (or use 5000 if needed)
# Install required libraries

In [7]:
!pip install requests PyPDF2 flask flask-cors pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00


In [ ]:
import requests
import PyPDF2
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import io

app = Flask(__name__)
CORS(app)

OLLAMA_API_URL = "http://localhost:11434/api/generate"

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_file.read()))
        text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
        return text.strip()
    except Exception as e:
        return f"Error extracting text: {str(e)}"

@app.route('/pdf/review-for-job', methods=['POST'])
def review_pdf_for_job():
    try:
        if 'resume' not in request.files or 'job_description' not in request.form:
            return jsonify({'error': 'Please upload a resume and provide a job description'}), 400

        pdf_file = request.files['resume']
        job_description = request.form['job_description'].strip()

        # Extract text from the uploaded PDF
        resume_text = extract_text_from_pdf(pdf_file)
        print(resume_text)

        if not resume_text:
            return jsonify({'error': 'Failed to extract text from the resume'}), 500

        # Prepare prompt for Ollama to analyze compatibility
        ollama_prompt = (
            f"Compare the following resume with the given job description:\n\n"
            f"Resume:\n{resume_text}\n\n"
            f"Job Description:\n{job_description}\n\n"
            f"1. Analyze compatibility between resume and job description.\n"
            f"2. Suggest areas where the candidate can improve their skills to match the job requirements.\n"
            f"Provide a short but structured response."
            f"Response should not be more than 7-8 lines ."
        )

        ollama_payload = {
            "model": "llama2",
            "prompt": ollama_prompt,
            "stream": False
        }

        # Send request to Ollama
        response = requests.post(OLLAMA_API_URL, json=ollama_payload)

        if response.status_code == 200:
            result = response.json()
            compatibility_analysis = result.get('response', 'No response received')

            return jsonify({
                'resume_text': resume_text,
                'job_description': job_description,
                'analysis': compatibility_analysis
            })
        else:
            return jsonify({'error': 'Failed to process resume for job review'}), 500

    except Exception as e:
        return jsonify({'error': str(e)}), 500

def start_ngrok():
    try:
        NGROK_AUTH_TOKEN = "2WG8G2YZNV9jofWBKu5B2E6hA3Q_719vVw9PLU1SwQLwCtcNd"
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        public_url = ngrok.connect(5000).public_url
        print(f"🚀 Public API URL: {public_url}")

    except Exception as e:
        print(f"Error setting up ngrok: {str(e)}")

def run_flask():
    app.run(port=5000, use_reloader=False)

if __name__ == '__main__':
    threading.Thread(target=start_ngrok).start()
    run_flask()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


🚀 Public API URL: https://995a-34-142-170-127.ngrok-free.app
